In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
# result = wiki.take(1)[0]


In [2]:
sc.uiWebUrl

u'http://172.17.0.3:4040'

In [3]:
KEYWORD = "narodnaya"

def find_ngrams(input_list):
    return zip(*[input_list[i:] for i in range(2)])

def ngram_filter(word_pair):
    return word_pair[0].lower() == KEYWORD

def printer(results):
    for pair in results[:50]:
        print "%s\t%i" % (pair[0], pair[1])




In [4]:
results = wiki.flatMap(find_ngrams) \
    .filter(ngram_filter) \
    .map(lambda pair: ("%s_%s" %(pair[0].lower(), pair[1].lower()), 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

In [5]:
printer(sorted(results))

narodnaya_gazeta	1
narodnaya_volya	9
